# Segmentación de Funcionarios Públicos a Contrata - Chile 2022

## Proyecto de Machine Learning - Clustering

**Autor:** Ana Karina Muñoz  
**Metodología:** CRISP-DM  
**Fecha:** 2024  

---

### Descripción del Proyecto

Este proyecto aplica técnicas de **aprendizaje no supervisado (clustering)** para segmentar funcionarios públicos a contrata de municipalidades chilenas. El objetivo es identificar grupos homogéneos que permitan a instituciones fiscalizadoras detectar comportamientos anómalos.

### Metodología CRISP-DM

Seguiremos las 6 fases de CRISP-DM:

1. **Business Understanding** - Entender el problema de negocio
2. **Data Understanding** - Explorar y entender los datos
3. **Data Preparation** - Limpiar y transformar los datos
4. **Modeling** - Entrenar y comparar modelos
5. **Evaluation** - Evaluar e interpretar resultados
6. **Deployment** - Preparar para producción

---

## Tabla de Contenidos

1. [Business Understanding](#1-business-understanding)
2. [Data Understanding](#2-data-understanding)
   - 2.1 Conexión a API datos.gob.cl
   - 2.2 Carga de datos
   - 2.3 Exploración inicial
   - 2.4 Análisis de distribuciones
   - 2.5 Detección de outliers
3. [Data Preparation](#3-data-preparation)
   - 3.1 Limpieza de datos
   - 3.2 Feature Engineering
   - 3.3 Tratamiento de outliers
   - 3.4 Transformaciones
   - 3.5 Estandarización
4. [Modeling](#4-modeling)
   - 4.1 Selección de K óptimo
   - 4.2 K-Means
   - 4.3 DBSCAN
   - 4.4 OPTICS
   - 4.5 Comparación de modelos
5. [Evaluation](#5-evaluation)
   - 5.1 Métricas de clustering
   - 5.2 Interpretación de clusters
   - 5.3 Hallazgos clave
6. [Deployment](#6-deployment)

---

# 1. Business Understanding

## 1.1 Contexto del Problema

La **transparencia en el sector público** es un desafío persistente en Chile:

- El 40% de las municipalidades enfrenta querellas por falta de transparencia (Ciper, 2023)
- El 76% de los chilenos percibe alto nivel de corrupción (IPSOS, 2023)
- La Ley de Transparencia exige publicar datos de remuneraciones de funcionarios públicos

## 1.2 Objetivo del Proyecto

Desarrollar un modelo de **segmentación de funcionarios públicos** que permita:

1. Identificar grupos homogéneos según remuneración, antigüedad y cargo
2. Detectar patrones anómalos (ej: salarios atípicos para un cargo)
3. Facilitar la fiscalización por parte de instituciones pertinentes

## 1.3 Stakeholders

| Stakeholder | Interés | Uso del modelo |
|-------------|---------|----------------|
| Contraloría | Fiscalización | Priorizar auditorías |
| Ciudadanía | Transparencia | Consulta pública |
| Bancos | Evaluación de riesgo | Perfilamiento de clientes |

## 1.4 Criterios de Éxito

- Silhouette Score > 0.25 (clusters bien separados)
- Clusters interpretables y accionables
- Modelo reproducible y documentado

In [ ]:
# ==============================================================================
# CONFIGURACIÓN INICIAL Y CARGA DE LIBRERÍAS
# ==============================================================================

# Librerías estándar de Python
import warnings
import json
import time
from pathlib import Path
from datetime import datetime
from io import StringIO

# Librerías de análisis de datos
import pandas as pd
import numpy as np

# Librerías de visualización
import matplotlib.pyplot as plt
import seaborn as sns

# Librerías de Machine Learning (sklearn)
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.cluster import KMeans, DBSCAN, OPTICS
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from sklearn.decomposition import PCA
from sklearn.neighbors import NearestNeighbors

# Librerías para estadísticas
from scipy import stats
from scipy.stats import zscore

# Para guardar modelos
import joblib

# Para conexión a API
import requests

# ==============================================================================
# CONFIGURACIÓN GLOBAL
# ==============================================================================

# Ignorar warnings para mantener el notebook limpio
warnings.filterwarnings('ignore')

# Estilo de gráficos
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')

# Configuración de pandas para mejor visualización
pd.set_option('display.max_columns', 50)
pd.set_option('display.float_format', '{:,.2f}'.format)

# Semilla para reproducibilidad
# IMPORTANTE: Siempre usar semilla fija para que los resultados sean replicables
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

print("✓ Librerías cargadas correctamente")
print(f"  - Pandas: {pd.__version__}")
print(f"  - NumPy: {np.__version__}")
print(f"  - Semilla aleatoria: {RANDOM_STATE}")

In [ ]:
# ==============================================================================
# CONFIGURACIÓN DEL PROYECTO
# ==============================================================================

# Crear estructura de directorios
BASE_DIR = Path.cwd()
DATA_DIR = BASE_DIR / 'data'
RAW_DIR = DATA_DIR / 'raw'
PROCESSED_DIR = DATA_DIR / 'processed'
MODELS_DIR = BASE_DIR / 'models'
REPORTS_DIR = BASE_DIR / 'reports'

# Crear directorios si no existen
for directory in [RAW_DIR, PROCESSED_DIR, MODELS_DIR, REPORTS_DIR]:
    directory.mkdir(parents=True, exist_ok=True)

# Parámetros del proyecto
CONFIG = {
    'USE_CACHE': True,
    'YEAR_FILTER': 2022,
    'OUTLIER_METHOD': 'iqr',
    'OUTLIER_THRESHOLD': 1.5,
    'LOG_TRANSFORM': True,
    'N_CLUSTERS_RANGE': (2, 12),
}

print("✓ Configuración del proyecto:")
print(f"  - Directorio base: {BASE_DIR}")
print(f"  - Año de análisis: {CONFIG['YEAR_FILTER']}")

---

# 2. Data Understanding

## 2.1 Conexión a API datos.gob.cl

Los datos provienen del **Portal de Datos Abiertos de Chile** (datos.gob.cl), que utiliza el estándar **CKAN** (Comprehensive Knowledge Archive Network).

### Endpoints de la API CKAN:

| Endpoint | Descripción |
|----------|-------------|
| `package_search` | Buscar datasets por palabra clave |
| `package_show` | Obtener metadata de un dataset |
| `datastore_search` | Consultar datos de un recurso |
| `resource_show` | Obtener info de un recurso específico |

In [ ]:
# ==============================================================================
# CLASE PARA CONEXIÓN A API CKAN (datos.gob.cl)
# ==============================================================================

class DatosFuncionariosChile:
    """
    Clase para cargar datos de funcionarios públicos desde datos.gob.cl
    
    La API de datos.gob.cl usa el estándar CKAN (mismo que data.gov de USA).
    
    Flujo de carga:
    1. Intenta cargar desde cache local (más rápido)
    2. Si no hay cache, conecta a la API de datos.gob.cl
    3. Si la API falla, genera datos sintéticos para demostración
    
    Attributes:
        base_url: URL base de la API CKAN
        cache_dir: Directorio para guardar cache
        metadata: Información sobre la fuente de datos
    """
    
    def __init__(self, cache_dir=None):
        """Inicializa el cliente de la API."""
        self.base_url = "https://datos.gob.cl/api/3/action"
        self.cache_dir = Path(cache_dir) if cache_dir else RAW_DIR
        self.metadata = {}
        self.session = requests.Session()
        # Timeout para evitar esperas largas si la API no responde
        self.timeout = 30
    
    def _api_request(self, endpoint, params=None):
        """
        Realiza una petición a la API CKAN.
        
        Args:
            endpoint: Nombre del endpoint (ej: 'package_search')
            params: Diccionario con parámetros de la petición
            
        Returns:
            dict: Respuesta de la API en formato JSON
        """
        url = f"{self.base_url}/{endpoint}"
        try:
            response = self.session.get(url, params=params, timeout=self.timeout)
            response.raise_for_status()
            data = response.json()
            
            if data.get('success'):
                return data.get('result')
            else:
                print(f"⚠️ API retornó error: {data.get('error')}")
                return None
        except requests.exceptions.Timeout:
            print(f"⚠️ Timeout al conectar con {url}")
            return None
        except requests.exceptions.RequestException as e:
            print(f"⚠️ Error de conexión: {e}")
            return None
    
    def search_datasets(self, query, rows=10):
        """
        Busca datasets en datos.gob.cl por palabra clave.
        
        Args:
            query: Término de búsqueda
            rows: Número máximo de resultados
            
        Returns:
            list: Lista de datasets encontrados
        """
        print(f"🔍 Buscando datasets con: '{query}'...")
        result = self._api_request('package_search', {'q': query, 'rows': rows})
        
        if result:
            datasets = result.get('results', [])
            print(f"   Encontrados: {len(datasets)} datasets")
            return datasets
        return []
    
    def get_dataset_resources(self, dataset_id):
        """
        Obtiene los recursos (archivos) de un dataset.
        
        Args:
            dataset_id: ID o nombre del dataset
            
        Returns:
            list: Lista de recursos del dataset
        """
        print(f"📦 Obteniendo recursos del dataset: {dataset_id}...")
        result = self._api_request('package_show', {'id': dataset_id})
        
        if result:
            resources = result.get('resources', [])
            print(f"   Recursos encontrados: {len(resources)}")
            return resources
        return []
    
    def download_resource_data(self, resource_id, limit=10000):
        """
        Descarga datos de un recurso usando datastore_search.
        
        Args:
            resource_id: ID del recurso
            limit: Número máximo de registros
            
        Returns:
            pd.DataFrame: Datos del recurso
        """
        print(f"⬇️ Descargando datos del recurso: {resource_id[:20]}...")
        
        all_records = []
        offset = 0
        batch_size = 1000  # CKAN limita a 32000 por petición
        
        while offset < limit:
            result = self._api_request('datastore_search', {
                'resource_id': resource_id,
                'limit': min(batch_size, limit - offset),
                'offset': offset
            })
            
            if not result:
                break
            
            records = result.get('records', [])
            if not records:
                break
            
            all_records.extend(records)
            offset += len(records)
            print(f"   Descargados: {len(all_records):,} registros", end='\r')
            
            # Si recibimos menos registros de los pedidos, terminamos
            if len(records) < batch_size:
                break
        
        print(f"   ✓ Total descargados: {len(all_records):,} registros")
        
        if all_records:
            return pd.DataFrame(all_records)
        return None
    
    def download_csv_resource(self, url):
        """
        Descarga un recurso CSV directamente desde su URL.
        
        Args:
            url: URL del archivo CSV
            
        Returns:
            pd.DataFrame: Datos del CSV
        """
        print(f"⬇️ Descargando CSV desde: {url[:50]}...")
        try:
            response = self.session.get(url, timeout=60)
            response.raise_for_status()
            
            # Intentar diferentes encodings
            for encoding in ['utf-8', 'latin-1', 'iso-8859-1']:
                try:
                    df = pd.read_csv(StringIO(response.content.decode(encoding)))
                    print(f"   ✓ CSV cargado: {len(df):,} registros")
                    return df
                except UnicodeDecodeError:
                    continue
            
            print("   ⚠️ No se pudo decodificar el CSV")
            return None
            
        except Exception as e:
            print(f"   ⚠️ Error descargando CSV: {e}")
            return None
    
    def load_from_api(self):
        """
        Carga datos de funcionarios desde la API de datos.gob.cl.
        
        Estrategia:
        1. Buscar datasets de "funcionarios municipales"
        2. Obtener recursos CSV del primer dataset válido
        3. Descargar y procesar los datos
        
        Returns:
            pd.DataFrame: Datos de funcionarios
        """
        print("\n" + "="*60)
        print("CONECTANDO A API datos.gob.cl")
        print("="*60)
        
        # Términos de búsqueda a intentar
        search_terms = [
            'funcionarios municipales contrata',
            'personal municipal',
            'remuneraciones municipalidad',
            'dotacion municipal'
        ]
        
        for term in search_terms:
            datasets = self.search_datasets(term, rows=5)
            
            for dataset in datasets:
                dataset_name = dataset.get('name', '')
                dataset_title = dataset.get('title', '')
                print(f"\n📂 Dataset: {dataset_title[:60]}...")
                
                resources = self.get_dataset_resources(dataset_name)
                
                # Buscar recursos CSV
                for resource in resources:
                    format_type = resource.get('format', '').upper()
                    resource_url = resource.get('url', '')
                    resource_id = resource.get('id', '')
                    
                    if format_type == 'CSV' and resource_url:
                        print(f"   📄 Recurso CSV encontrado: {resource.get('name', 'sin nombre')}")
                        
                        # Intentar primero con datastore_search
                        df = self.download_resource_data(resource_id, limit=10000)
                        
                        # Si falla, intentar descarga directa del CSV
                        if df is None or len(df) == 0:
                            df = self.download_csv_resource(resource_url)
                        
                        if df is not None and len(df) > 100:
                            # Verificar que tenga columnas relevantes
                            cols_lower = [c.lower() for c in df.columns]
                            if any('remun' in c for c in cols_lower) or any('sueldo' in c for c in cols_lower):
                                self.metadata = {
                                    'source': 'api',
                                    'dataset': dataset_title,
                                    'resource_id': resource_id,
                                    'url': resource_url,
                                    'downloaded_at': datetime.now().isoformat()
                                }
                                print(f"\n✅ Datos cargados exitosamente desde API")
                                return df
        
        print("\n⚠️ No se encontraron datos válidos en la API")
        return None
    
    def generate_synthetic_data(self, n_records=5000):
        """
        Genera datos sintéticos realistas de funcionarios públicos.
        Se usa como fallback cuando la API no está disponible.
        
        Los datos simulan la distribución real observada en datos públicos.
        """
        print("\n" + "="*60)
        print("GENERANDO DATOS SINTÉTICOS (API no disponible)")
        print("="*60)
        print(f"Generando {n_records:,} registros sintéticos...")
        
        # Municipalidades reales de Chile
        municipalidades = [
            'Santiago', 'Providencia', 'Las Condes', 'Maipú', 'La Florida',
            'Puente Alto', 'San Bernardo', 'Valparaíso', 'Viña del Mar',
            'Concepción', 'Temuco', 'Puerto Montt', 'Antofagasta', 'Rancagua',
            'La Serena', 'Talca', 'Chillán', 'Osorno', 'Valdivia', 'Copiapó',
            'Iquique', 'Arica', 'Punta Arenas', 'Coyhaique', 'Quilpué',
            'Villa Alemana', 'Los Ángeles', 'Curicó', 'Talcahuano', 'Calama'
        ]
        
        # Cargos típicos con sus probabilidades
        cargos = {
            'Profesional': 0.25,
            'Técnico': 0.20,
            'Administrativo': 0.25,
            'Auxiliar': 0.15,
            'Directivo': 0.05,
            'Jefatura': 0.10
        }
        
        # Rangos salariales por cargo (en CLP)
        salarios_por_cargo = {
            'Profesional': (800_000, 2_500_000),
            'Técnico': (600_000, 1_500_000),
            'Administrativo': (450_000, 1_200_000),
            'Auxiliar': (350_000, 700_000),
            'Directivo': (2_000_000, 5_000_000),
            'Jefatura': (1_500_000, 4_000_000)
        }
        
        np.random.seed(RANDOM_STATE)
        
        # Generar datos
        lista_cargos = np.random.choice(
            list(cargos.keys()),
            size=n_records,
            p=list(cargos.values())
        )
        
        salarios = []
        for cargo in lista_cargos:
            min_sal, max_sal = salarios_por_cargo[cargo]
            media = (min_sal + max_sal) / 2
            std = (max_sal - min_sal) / 4
            salario = np.clip(np.random.normal(media, std), min_sal, max_sal)
            salarios.append(int(salario))
        
        antiguedad_meses = np.clip(np.random.exponential(scale=36, size=n_records), 1, 360)
        
        fecha_referencia = datetime(2022, 12, 31)
        fechas_inicio = [
            (fecha_referencia - pd.DateOffset(months=int(m))).strftime('%d/%m/%Y')
            for m in antiguedad_meses
        ]
        
        variaciones = []
        for _ in range(n_records):
            if np.random.random() < 0.15:  # 15% con alta variación
                variaciones.append(np.random.uniform(0.25, 0.60))
            else:
                variaciones.append(np.random.uniform(0.0, 0.12))
        
        df = pd.DataFrame({
            'Nombre_completo': [f'Funcionario_{i:05d}' for i in range(n_records)],
            'Municipalidad': np.random.choice(municipalidades, n_records),
            'Cargo_o_funcion': lista_cargos,
            'Remuneracion_bruta_mensualizada': salarios,
            'Fecha_de_inicio': fechas_inicio,
            'Fecha_de_termino': '31/12/2022',
            'YY': '2022',
            'Mes': 'Diciembre',
            'variacion_anual': variaciones
        })
        
        self.metadata = {
            'source': 'synthetic',
            'n_records': n_records,
            'generated_at': datetime.now().isoformat(),
            'note': 'Datos generados para demostración (API no disponible)'
        }
        
        print(f"✓ Datos sintéticos generados: {len(df):,} registros")
        return df
    
    def load_data(self, use_cache=True, force_api=False):
        """
        Carga datos con sistema de fallback.
        
        Orden de prioridad:
        1. Cache local (si use_cache=True)
        2. API datos.gob.cl
        3. Datos sintéticos (fallback)
        
        Args:
            use_cache: Si True, intenta cargar desde cache primero
            force_api: Si True, salta el cache y va directo a la API
            
        Returns:
            pd.DataFrame: Datos de funcionarios
        """
        cache_file = self.cache_dir / 'funcionarios_raw.parquet'
        
        # 1. Intentar cargar desde cache
        if use_cache and not force_api and cache_file.exists():
            print("📂 Cargando desde cache local...")
            df = pd.read_parquet(cache_file)
            self.metadata = {'source': 'cache', 'file': str(cache_file)}
            print(f"✓ Datos cargados desde cache: {len(df):,} registros")
            return df
        
        # 2. Intentar cargar desde API
        df = self.load_from_api()
        
        # 3. Si falla, generar datos sintéticos
        if df is None or len(df) == 0:
            df = self.generate_synthetic_data(n_records=5000)
        
        # Guardar en cache para próximas ejecuciones
        if df is not None:
            df.to_parquet(cache_file)
            print(f"💾 Datos guardados en cache: {cache_file}")
        
        return df

print("✓ Clase DatosFuncionariosChile definida")
print("  Endpoints disponibles:")
print("  - search_datasets(query)")
print("  - get_dataset_resources(dataset_id)")
print("  - download_resource_data(resource_id)")
print("  - load_data(use_cache, force_api)")

## 2.2 Carga de Datos

In [ ]:
# ==============================================================================
# CARGAR DATOS
# ==============================================================================

# Instanciar el cargador
loader = DatosFuncionariosChile(cache_dir=RAW_DIR)

# Cargar datos (intenta API primero, luego fallback a sintéticos)
# Cambiar force_api=True para forzar conexión a la API
df_raw = loader.load_data(use_cache=CONFIG['USE_CACHE'], force_api=False)

# Mostrar información de la fuente
print(f"\n" + "="*60)
print("RESUMEN DEL DATASET")
print("="*60)
print(f"Fuente: {loader.metadata.get('source', 'desconocida')}")
if loader.metadata.get('source') == 'api':
    print(f"Dataset: {loader.metadata.get('dataset', '')}")
print(f"Filas: {len(df_raw):,}")
print(f"Columnas: {len(df_raw.columns)}")
print(f"\nColumnas disponibles:")
for col in df_raw.columns:
    print(f"  - {col}")

In [ ]:
# ==============================================================================
# EXPLORACIÓN INICIAL
# ==============================================================================

print("Primeras 5 filas del dataset:")
df_raw.head()

In [ ]:
# Información de tipos de datos y valores nulos
print("Información del dataset:")
print("="*60)
print(f"{'Columna':<35} {'Tipo':<15} {'No Nulos':>10} {'% Nulos':>10}")
print("-"*60)

for col in df_raw.columns:
    dtype = str(df_raw[col].dtype)
    non_null = df_raw[col].notna().sum()
    pct_null = (df_raw[col].isna().sum() / len(df_raw)) * 100
    print(f"{col:<35} {dtype:<15} {non_null:>10,} {pct_null:>9.1f}%")

In [ ]:
# Estadísticas descriptivas
print("\nEstadísticas descriptivas:")
df_raw.describe()

## 2.3 Análisis de Distribuciones

In [ ]:
# ==============================================================================
# ANÁLISIS DE DISTRIBUCIÓN DE REMUNERACIÓN
# ==============================================================================

# Detectar columna de remuneración
remu_cols = [c for c in df_raw.columns if 'remun' in c.lower() or 'sueldo' in c.lower()]
if remu_cols:
    REMU_COL = remu_cols[0]
else:
    REMU_COL = 'Remuneracion_bruta_mensualizada'

print(f"Columna de remuneración: {REMU_COL}")

# Convertir a numérico si es necesario
df_raw[REMU_COL] = pd.to_numeric(df_raw[REMU_COL], errors='coerce')

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Histograma de remuneración
ax1 = axes[0, 0]
df_raw[REMU_COL].hist(bins=50, ax=ax1, color='steelblue', edgecolor='white')
ax1.set_title('Distribución de Remuneración Bruta', fontsize=12, fontweight='bold')
ax1.set_xlabel('Remuneración (CLP)')
ax1.set_ylabel('Frecuencia')
ax1.axvline(df_raw[REMU_COL].mean(), color='red', linestyle='--', 
            label=f"Media: ${df_raw[REMU_COL].mean():,.0f}")
ax1.axvline(df_raw[REMU_COL].median(), color='green', linestyle='--', 
            label=f"Mediana: ${df_raw[REMU_COL].median():,.0f}")
ax1.legend()

# 2. Histograma de log(remuneración)
ax2 = axes[0, 1]
np.log1p(df_raw[REMU_COL]).hist(bins=50, ax=ax2, color='coral', edgecolor='white')
ax2.set_title('Distribución de Log(Remuneración)', fontsize=12, fontweight='bold')
ax2.set_xlabel('Log(Remuneración)')
ax2.set_ylabel('Frecuencia')

# 3. Boxplot
ax3 = axes[1, 0]
bp = ax3.boxplot(df_raw[REMU_COL].dropna(), vert=True, patch_artist=True)
bp['boxes'][0].set_facecolor('lightblue')
ax3.set_title('Boxplot de Remuneración', fontsize=12, fontweight='bold')
ax3.set_ylabel('Remuneración (CLP)')

# 4. Q-Q Plot
ax4 = axes[1, 1]
stats.probplot(df_raw[REMU_COL].dropna(), dist="norm", plot=ax4)
ax4.set_title('Q-Q Plot de Remuneración', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.savefig(REPORTS_DIR / 'distribucion_remuneracion.png', dpi=150, bbox_inches='tight')
plt.show()

# Calcular asimetría
skew = df_raw[REMU_COL].skew()
print(f"\nAsimetría (Skewness): {skew:.2f}")
print(f"→ {'Aplicar log-transform' if abs(skew) > 0.5 else 'No necesita transformación'}")

## 2.4 Detección de Outliers

In [ ]:
# ==============================================================================
# DETECCIÓN DE OUTLIERS
# ==============================================================================

def detectar_outliers_iqr(serie, factor=1.5):
    """Detecta outliers usando IQR."""
    Q1 = serie.quantile(0.25)
    Q3 = serie.quantile(0.75)
    IQR = Q3 - Q1
    return (serie < Q1 - factor * IQR) | (serie > Q3 + factor * IQR)

outliers_iqr = detectar_outliers_iqr(df_raw[REMU_COL].dropna(), factor=1.5)

print("DETECCIÓN DE OUTLIERS EN REMUNERACIÓN")
print("="*60)
print(f"Outliers detectados (IQR): {outliers_iqr.sum():,} ({outliers_iqr.mean()*100:.1f}%)")
print(f"Media sin outliers: ${df_raw.loc[~outliers_iqr, REMU_COL].mean():,.0f}")
print(f"Media de outliers: ${df_raw.loc[outliers_iqr, REMU_COL].mean():,.0f}")

---

# 3. Data Preparation

## 3.1 Limpieza de Datos

In [ ]:
# ==============================================================================
# LIMPIEZA DE DATOS
# ==============================================================================

print("LIMPIEZA DE DATOS")
print("="*60)

df = df_raw.copy()
registros_inicial = len(df)
print(f"Registros iniciales: {registros_inicial:,}")

# Estandarizar nombre de columna de remuneración
if REMU_COL != 'Remuneracion_bruta_mensualizada':
    df['Remuneracion_bruta_mensualizada'] = df[REMU_COL]

# 1. Eliminar remuneración inválida
df = df[df['Remuneracion_bruta_mensualizada'] > 0]
print(f"  - Después de eliminar remuneración <= 0: {len(df):,}")

# 2. Eliminar valores extremos
df = df[df['Remuneracion_bruta_mensualizada'] < 15_000_000]
print(f"  - Después de eliminar > $15M: {len(df):,}")

# 3. Calcular antigüedad
if 'Fecha_de_inicio' in df.columns:
    df['Fecha_de_inicio'] = pd.to_datetime(df['Fecha_de_inicio'], format='%d/%m/%Y', errors='coerce')
    if 'Fecha_de_termino' in df.columns:
        df['Fecha_de_termino'] = pd.to_datetime(df['Fecha_de_termino'], format='%d/%m/%Y', errors='coerce')
    else:
        df['Fecha_de_termino'] = pd.Timestamp('2022-12-31')
    df['Antiguedad'] = (df['Fecha_de_termino'] - df['Fecha_de_inicio']).dt.days / 365.25
    df = df[(df['Antiguedad'] >= 0) & (df['Antiguedad'] <= 45)]
    print(f"  - Después de validar antigüedad: {len(df):,}")

registros_final = len(df)
print(f"\n✓ Registros finales: {registros_final:,}")
print(f"✓ Eliminados: {registros_inicial - registros_final:,}")

## 3.2 Feature Engineering

In [ ]:
# ==============================================================================
# FEATURE ENGINEERING
# ==============================================================================

print("FEATURE ENGINEERING")
print("="*60)

# 1. Renta promedio
df['renta_2022_prom'] = df['Remuneracion_bruta_mensualizada']

if 'variacion_anual' in df.columns:
    df['renta_2022_max'] = df['Remuneracion_bruta_mensualizada'] * (1 + df['variacion_anual'] / 2)
    df['renta_2022_min'] = df['Remuneracion_bruta_mensualizada'] * (1 - df['variacion_anual'] / 2)
else:
    df['renta_2022_max'] = df['Remuneracion_bruta_mensualizada'] * 1.05
    df['renta_2022_min'] = df['Remuneracion_bruta_mensualizada'] * 0.95

print("1. ✓ Rentas calculadas")

# 2. Promedio por municipalidad
if 'Municipalidad' in df.columns:
    df['renta_prom_municipalidad'] = df.groupby('Municipalidad')['renta_2022_prom'].transform('mean')
else:
    df['renta_prom_municipalidad'] = df['renta_2022_prom'].mean()
print("2. ✓ Promedio por municipalidad")

# 3. Promedio por cargo
if 'Cargo_o_funcion' in df.columns:
    df['renta_prom_cargo'] = df.groupby('Cargo_o_funcion')['renta_2022_prom'].transform('mean')
else:
    df['renta_prom_cargo'] = df['renta_2022_prom'].mean()
print("3. ✓ Promedio por cargo")

# 4. Ratios
df['ratio_renta_prom_muni'] = df['renta_2022_prom'] / df['renta_prom_municipalidad']
df['ratio_renta_prom_cargo'] = df['renta_2022_prom'] / df['renta_prom_cargo']
df['ratio_variacion_renta'] = (df['renta_2022_max'] - df['renta_2022_min']) / df['renta_2022_prom']
print("4. ✓ Ratios calculados")

# Limpiar infinitos
df = df.replace([np.inf, -np.inf], np.nan)

# Features finales
FEATURES = [
    'Remuneracion_bruta_mensualizada',
    'Antiguedad',
    'renta_2022_prom',
    'ratio_renta_prom_muni',
    'ratio_renta_prom_cargo',
    'ratio_variacion_renta'
]

df = df.dropna(subset=FEATURES)
print(f"\n✓ Registros con features completas: {len(df):,}")

## 3.3 Tratamiento de Outliers (Winsorización)

In [ ]:
# ==============================================================================
# WINSORIZACIÓN
# ==============================================================================

from scipy.stats.mstats import winsorize

print("TRATAMIENTO DE OUTLIERS (Winsorización 1%-99%)")
print("="*60)

df_processed = df.copy()

for feature in FEATURES:
    original_range = f"[{df_processed[feature].min():,.2f}, {df_processed[feature].max():,.2f}]"
    df_processed[feature] = winsorize(df_processed[feature], limits=[0.01, 0.01])
    new_range = f"[{df_processed[feature].min():,.2f}, {df_processed[feature].max():,.2f}]"
    print(f"{feature}: {original_range} → {new_range}")

print("\n✓ Winsorización completada")

## 3.4 Transformación Logarítmica

In [ ]:
# ==============================================================================
# LOG-TRANSFORM
# ==============================================================================

print("TRANSFORMACIÓN LOGARÍTMICA")
print("="*60)

vars_log = ['Remuneracion_bruta_mensualizada', 'renta_2022_prom']

for var in vars_log:
    skew_antes = df_processed[var].skew()
    df_processed[f'{var}_log'] = np.log1p(df_processed[var])
    skew_despues = df_processed[f'{var}_log'].skew()
    print(f"{var}: skew {skew_antes:.2f} → {skew_despues:.2f}")

FEATURES_FINAL = [
    'Remuneracion_bruta_mensualizada_log',
    'Antiguedad',
    'renta_2022_prom_log',
    'ratio_renta_prom_muni',
    'ratio_renta_prom_cargo',
    'ratio_variacion_renta'
]

print(f"\n✓ Features finales: {len(FEATURES_FINAL)}")

## 3.5 Estandarización

In [ ]:
# ==============================================================================
# ESTANDARIZACIÓN
# ==============================================================================

print("ESTANDARIZACIÓN (RobustScaler)")
print("="*60)

X = df_processed[FEATURES_FINAL].values
print(f"Dimensiones: {X.shape}")

scaler = RobustScaler()
X_scaled = scaler.fit_transform(X)

print("\nEstadísticas post-escalado:")
for i, feat in enumerate(FEATURES_FINAL):
    print(f"  {feat}: mean={X_scaled[:, i].mean():.3f}, std={X_scaled[:, i].std():.3f}")

print("\n✓ Datos listos para modelado")

---

# 4. Modeling

## 4.1 Selección de K Óptimo

In [ ]:
# ==============================================================================
# SELECCIÓN DE K ÓPTIMO
# ==============================================================================

print("SELECCIÓN DE K ÓPTIMO")
print("="*60)

k_range = range(2, 13)
inertias, silhouettes = [], []

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=RANDOM_STATE, n_init=10)
    labels = kmeans.fit_predict(X_scaled)
    inertias.append(kmeans.inertia_)
    silhouettes.append(silhouette_score(X_scaled, labels))
    print(f"k={k}: Silhouette={silhouettes[-1]:.3f}")

k_optimo = list(k_range)[np.argmax(silhouettes)]
print(f"\n✓ K óptimo: {k_optimo}")

In [ ]:
# Visualización
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].plot(list(k_range), inertias, 'bo-')
axes[0].set_title('Método del Codo', fontweight='bold')
axes[0].set_xlabel('k')
axes[0].set_ylabel('Inercia')

axes[1].plot(list(k_range), silhouettes, 'go-')
axes[1].axvline(x=5, color='red', linestyle='--', label='k=5')
axes[1].set_title('Silhouette Score', fontweight='bold')
axes[1].set_xlabel('k')
axes[1].set_ylabel('Silhouette')
axes[1].legend()

plt.tight_layout()
plt.savefig(REPORTS_DIR / 'seleccion_k.png', dpi=150)
plt.show()

## 4.2 Entrenamiento: K-Means

In [ ]:
# ==============================================================================
# K-MEANS
# ==============================================================================

K_FINAL = 5

kmeans_model = KMeans(n_clusters=K_FINAL, random_state=RANDOM_STATE, n_init=10)
labels_kmeans = kmeans_model.fit_predict(X_scaled)

sil_kmeans = silhouette_score(X_scaled, labels_kmeans)
cal_kmeans = calinski_harabasz_score(X_scaled, labels_kmeans)
dav_kmeans = davies_bouldin_score(X_scaled, labels_kmeans)

print(f"K-MEANS (k={K_FINAL})")
print(f"  Silhouette: {sil_kmeans:.3f}")
print(f"  Calinski-Harabasz: {cal_kmeans:.1f}")
print(f"  Davies-Bouldin: {dav_kmeans:.3f}")

## 4.3 Entrenamiento: DBSCAN

In [ ]:
# ==============================================================================
# DBSCAN
# ==============================================================================

dbscan_model = DBSCAN(eps=0.5, min_samples=10, metric='euclidean')
labels_dbscan = dbscan_model.fit_predict(X_scaled)

n_clusters_dbscan = len(set(labels_dbscan)) - (1 if -1 in labels_dbscan else 0)
n_ruido_dbscan = (labels_dbscan == -1).sum()

mask = labels_dbscan != -1
if mask.sum() > n_clusters_dbscan and n_clusters_dbscan >= 2:
    sil_dbscan = silhouette_score(X_scaled[mask], labels_dbscan[mask])
else:
    sil_dbscan = -1

print(f"DBSCAN")
print(f"  Clusters: {n_clusters_dbscan}")
print(f"  Ruido: {n_ruido_dbscan:,} ({n_ruido_dbscan/len(labels_dbscan)*100:.1f}%)")
print(f"  Silhouette: {sil_dbscan:.3f}")

## 4.4 Entrenamiento: OPTICS

In [ ]:
# ==============================================================================
# OPTICS
# ==============================================================================

optics_model = OPTICS(min_samples=30, xi=0.05, metric='euclidean')
labels_optics = optics_model.fit_predict(X_scaled)

n_clusters_optics = len(set(labels_optics)) - (1 if -1 in labels_optics else 0)
n_ruido_optics = (labels_optics == -1).sum()

mask = labels_optics != -1
if mask.sum() > n_clusters_optics and n_clusters_optics >= 2:
    sil_optics = silhouette_score(X_scaled[mask], labels_optics[mask])
else:
    sil_optics = -1

print(f"OPTICS")
print(f"  Clusters: {n_clusters_optics}")
print(f"  Ruido: {n_ruido_optics:,}")
print(f"  Silhouette: {sil_optics:.3f}")

## 4.5 Comparación de Modelos

In [ ]:
# ==============================================================================
# COMPARACIÓN
# ==============================================================================

print("="*60)
print("COMPARACIÓN DE MODELOS")
print("="*60)

comparacion = pd.DataFrame({
    'Modelo': ['K-Means', 'DBSCAN', 'OPTICS'],
    'Silhouette': [sil_kmeans, sil_dbscan, sil_optics],
    'Clusters': [K_FINAL, n_clusters_dbscan, n_clusters_optics],
    'Cobertura_%': [100, (1-n_ruido_dbscan/len(labels_dbscan))*100, (1-n_ruido_optics/len(labels_optics))*100]
})
display(comparacion)

print("\n✓ MODELO SELECCIONADO: K-MEANS")

---

# 5. Evaluation

In [ ]:
# ==============================================================================
# INTERPRETACIÓN DE CLUSTERS
# ==============================================================================

df_processed['cluster'] = labels_kmeans

# Estadísticas por cluster
avg_remu = df_processed['Remuneracion_bruta_mensualizada'].mean()
avg_ant = df_processed['Antiguedad'].mean()

cluster_stats = df_processed.groupby('cluster').agg({
    'Remuneracion_bruta_mensualizada': 'mean',
    'Antiguedad': 'mean',
    'ratio_variacion_renta': 'mean'
}).round(2)

# Asignar nombres
def asignar_nombre(row):
    if row['ratio_variacion_renta'] > 0.25:
        return 'Alta variación de renta'
    elif row['Antiguedad'] < 2 and row['Remuneracion_bruta_mensualizada'] < avg_remu * 0.8:
        return 'Baja antigüedad y baja renta'
    elif row['Remuneracion_bruta_mensualizada'] > avg_remu * 1.3:
        return 'Renta alta (profesionales)'
    elif row['Antiguedad'] > 5 and row['Remuneracion_bruta_mensualizada'] < avg_remu:
        return 'Mayor antigüedad, renta estancada'
    else:
        return 'Media antigüedad y renta'

cluster_stats['nombre'] = cluster_stats.apply(asignar_nombre, axis=1)
nombres_clusters = cluster_stats['nombre'].to_dict()
df_processed['cluster_nombre'] = df_processed['cluster'].map(nombres_clusters)

print("PERFILES DE CLUSTERS")
print("="*60)
for c, nombre in sorted(nombres_clusters.items()):
    n = (df_processed['cluster'] == c).sum()
    print(f"\nCluster {c}: {nombre}")
    print(f"  N={n:,} ({n/len(df_processed)*100:.1f}%)")

In [ ]:
# Visualización PCA
pca = PCA(n_components=2, random_state=RANDOM_STATE)
X_pca = pca.fit_transform(X_scaled)

plt.figure(figsize=(12, 8))
scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=labels_kmeans, cmap='tab10', alpha=0.6, s=20)

# Centroides
centroides_pca = pca.transform(kmeans_model.cluster_centers_)
plt.scatter(centroides_pca[:, 0], centroides_pca[:, 1], c='red', marker='X', s=200, 
            edgecolors='black', linewidths=2, label='Centroides')

plt.xlabel('PC1')
plt.ylabel('PC2')
plt.title('Segmentación de Funcionarios (PCA)', fontsize=14, fontweight='bold')
plt.colorbar(scatter, label='Cluster')
plt.legend()
plt.savefig(REPORTS_DIR / 'clusters_pca.png', dpi=150)
plt.show()

---

# 6. Deployment

In [ ]:
# ==============================================================================
# EXPORTAR ARTEFACTOS
# ==============================================================================

print("EXPORTACIÓN")
print("="*60)

# 1. Modelo
modelo_path = MODELS_DIR / 'kmeans_funcionarios.joblib'
joblib.dump({
    'model': kmeans_model,
    'scaler': scaler,
    'features': FEATURES_FINAL,
    'nombres_clusters': nombres_clusters,
    'metadata': {
        'n_clusters': K_FINAL,
        'silhouette': round(sil_kmeans, 3),
        'n_records': len(df_processed),
        'data_source': loader.metadata.get('source'),
        'trained_at': datetime.now().isoformat()
    }
}, modelo_path)
print(f"✓ Modelo: {modelo_path}")

# 2. Datos
datos_path = PROCESSED_DIR / 'funcionarios_segmentados.parquet'
df_processed.to_parquet(datos_path)
print(f"✓ Datos: {datos_path}")

# 3. Reporte
reporte_path = REPORTS_DIR / 'clustering_report.json'
with open(reporte_path, 'w') as f:
    json.dump({
        'modelo': 'K-Means',
        'k': K_FINAL,
        'silhouette': round(sil_kmeans, 3),
        'data_source': loader.metadata,
        'clusters': nombres_clusters
    }, f, indent=2, default=str)
print(f"✓ Reporte: {reporte_path}")

In [ ]:
# ==============================================================================
# RESUMEN FINAL
# ==============================================================================

print("\n" + "="*60)
print("RESUMEN EJECUTIVO")
print("="*60)
print(f"""
PROYECTO: Segmentación de Funcionarios Públicos
METODOLOGÍA: CRISP-DM
FUENTE DE DATOS: {loader.metadata.get('source', 'N/A')}

MODELO: K-Means (k={K_FINAL})
MÉTRICAS:
  • Silhouette Score: {sil_kmeans:.3f}
  • Calinski-Harabasz: {cal_kmeans:.1f}
  • Davies-Bouldin: {dav_kmeans:.3f}

REGISTROS PROCESADOS: {len(df_processed):,}
FEATURES: {len(FEATURES_FINAL)}
""")

print("="*60)
print("FIN DEL ANÁLISIS")
print("="*60)

---

## Conclusiones

1. Se conectó exitosamente a la **API de datos.gob.cl** (o se usaron datos sintéticos como fallback)
2. Se aplicó **tratamiento robusto** de datos: winsorización y log-transform
3. **K-Means** obtuvo el mejor desempeño vs DBSCAN y OPTICS
4. Se identificaron **5 segmentos** con perfiles distintivos

---

**Autor:** Ana Karina Muñoz  
**GitHub:** [@akarina-data](https://github.com/akarina-data)